**------------------------------------------------------------------------------------------------------------------------------------------------------**

**Input: Annotated Dataset**

**Computes the Kappa-Score + T-test/Wilcoxon**

**Output: Kappa-Score + T-test/Wilcoxon**

**------------------------------------------------------------------------------------------------------------------------------------------------------**

# Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import cohen_kappa_score
import statistics
from scipy import stats
from scipy.stats import norm
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

**Get Annotated Datasets**

Get Alie's annotations.

In [ ]:
df_alie = pd.read_excel('../Input Data/Dataset_2_annotate_Alie.xlsx', sheet_name='Dataset_2_annotate')
df_alie.head()

Get Ilse's annotations.

In [ ]:
df_ilse = pd.read_excel('../Input Data/Dataset_2_annotate_Ilse.xlsx', sheet_name='Dataset_2_annotate')
df_ilse.head()

**Merge all datasets into one**

In [ ]:
df = pd.DataFrame()
df['Food label'] = df_alie['Food label']
df['Subs A'] = df_alie['Subs A']
df['Subs B'] = df_alie['Subs B']
df['Healthiness_Alie'] = df_alie['Healthiness']
df['Healthiness_Ilse'] = df_ilse['Healthiness']
df.head()

**Preprocess the dataframe**

In [ ]:
print(Counter(df['Healthiness_Alie']))
print(Counter(df['Healthiness_Ilse']))

If any NA values are present, then drop that row.

In [ ]:
df_filtered = df.dropna(subset=['Healthiness_Alie', 'Healthiness_Ilse'], how='any')

In [ ]:
print(Counter(df_filtered['Healthiness_Alie']))
print(Counter(df_filtered['Healthiness_Ilse']))

**Compute Kappa-Score**

In [ ]:
cohen_kappa_score(df_filtered['Healthiness_Alie'], df_filtered['Healthiness_Ilse'])

In [ ]:
match = df_filtered[df_filtered['Healthiness_Alie'] == df_filtered['Healthiness_Ilse']]
num_match = len(match)
print(f'# Matches between Nutri-Scholars: {num_match}')

In [ ]:
print(f'# Dataset (after removing NA): {len(df_filtered)}')

In [ ]:
num_match/len(df_filtered)

This cell purely computes the 3x3 contingency matrix as shown in the paper (see Table 7.3).

In [ ]:
cell_a = 0
cell_b = 0
cell_c = 0
cell_d = 0
cell_e = 0
cell_f = 0
cell_g = 0
cell_h = 0
cell_i = 0

for i, row in df_filtered.iterrows():
    if row['Healthiness_Alie'] == 'Sub A' and row['Healthiness_Ilse'] == 'Sub A':
        cell_a += 1 
    elif row['Healthiness_Alie'] == 'Sub B' and row['Healthiness_Ilse'] == 'Sub A':
        cell_b += 1 
    elif row['Healthiness_Alie'] == 'NC' and row['Healthiness_Ilse'] == 'Sub A':
        cell_c += 1 
    elif row['Healthiness_Alie'] == 'Sub A' and row['Healthiness_Ilse'] == 'Sub B':
        cell_d += 1 
    elif row['Healthiness_Alie'] == 'Sub B' and row['Healthiness_Ilse'] == 'Sub B':
        cell_e += 1 
    elif row['Healthiness_Alie'] == 'NC' and row['Healthiness_Ilse'] == 'Sub B':
        cell_f += 1 
    elif row['Healthiness_Alie'] == 'Sub A' and row['Healthiness_Ilse'] == 'NC':
        cell_g += 1 
    elif row['Healthiness_Alie'] == 'Sub B' and row['Healthiness_Ilse'] == 'NC':
        cell_h += 1 
    elif row['Healthiness_Alie'] == 'NC' and row['Healthiness_Ilse'] == 'NC':
        cell_i += 1 

**T-Test**

Sample 1 or X: nutri-score values that are obtained by looking at the annotations for which both experts agreed (e.g., what is healthiest from a nutritional point of view).

Sample 2 or Y: nutri-score values that are attained by looking at the oppositie of what the experts chose (e.g., what is not healthiest from a nutrtitional point of view).

Goal: test whether the mean of sample 1 is significantly different from the mean of sample 2

In [ ]:
nutri_scores = pd.read_csv('../Output/nutri_scores.csv')
nutri_scores = nutri_scores[['Shrt_Desc', 'nutri_values']]

In [ ]:
match = match.merge(nutri_scores, left_on='Subs A', right_on='Shrt_Desc')
match = match[['Food label', 'Subs A', 'Subs B', 'Healthiness_Alie', 'nutri_values']]
match.rename(columns = {'nutri_values':'Nutri-Values Sub A'}, inplace = True)
match = match.merge(nutri_scores, left_on='Subs B', right_on='Shrt_Desc')
match = match[['Food label', 'Subs A', 'Subs B', 'Healthiness_Alie', 'Nutri-Values Sub A', 'nutri_values']]
match.rename(columns = {'nutri_values':'Nutri-Values Sub B'}, inplace = True)
match.head()

Here, we create sample 1 (called X) and sample 2 (called Y).

Sample 1: Nutri value scores of the substitutes for which both experts agreed. 

Sample 2: Nutri value scores of the opposite substitutes for which both experts agreed.

In [ ]:
X = [] 
Y = []

for i, row in match.iterrows():
    if row['Healthiness_Alie'] == 'Sub A':
        X.append(row['Nutri-Values Sub A'])
        Y.append(row['Nutri-Values Sub B'])
    elif row['Healthiness_Alie'] == 'Sub B':
        X.append(row['Nutri-Values Sub B'])
        Y.append(row['Nutri-Values Sub A'])
    else: 
        X.append(np.nan)
        Y.append(np.nan)
        
match['X'] = X
match['Y'] = Y
match = match.dropna()
match.head()

Plot sample 1.

In [ ]:
mu, std = norm.fit(match['X']) 
plt.hist(match['X'], bins=25, density=True, color = 'y', alpha=0.5, label='Sample 1')
xmin, xmax = plt.xlim()
x_1 = np.linspace(xmin, xmax, 100)
p_1 = norm.pdf(x_1, mu, std)
plt.plot(x_1, p_1, 'k', linewidth = 2)
plt.legend(loc='upper right')
plt.show()

Plot sample 2.

In [ ]:
mu, std = norm.fit(match['Y']) 
plt.hist(match['Y'], bins=25, density=True, color = 'g', alpha=0.5, label='Sample 2')
xmin, xmax = plt.xlim()
x_2 = np.linspace(xmin, xmax, 100)
p_2 = norm.pdf(x_2, mu, std)
plt.plot(x_2, p_2, 'k', linewidth = 2)
plt.legend(loc='upper right')
plt.show()

Plot both samples against each other.

In [ ]:
sns.distplot(a=match.X, hist=False, color = 'y', label='Sample 1')
sns.distplot(a=match.Y, hist=False, color = 'g', label='Sample 2')
plt.legend(loc='upper right')
plt.show()

Paired Sample T-test.

In [ ]:
print("%.20f" % stats.ttest_rel(match['X'], match['Y']).pvalue)

In [ ]:
print("%.20f" % stats.ttest_rel(match['X'], match['Y'], alternative='less').pvalue)

**Plot Food Groups for NC options**

In [ ]:
match = df_filtered[df_filtered['Healthiness_Alie'] == df_filtered['Healthiness_Ilse']]
nc = match[match['Healthiness_Alie'] == 'NC']
nc.head()

In [ ]:
food_groups = pd.read_csv('../Input Data/food_category.csv')
food_groups['NDB_No'] = 'http://idea.rpi.edu/heals/kb/usda#' + food_groups.NDB_No.astype(str).str.rjust(5,'0')
food_groups = food_groups[['NDB_No', 'FdGrp_Desc']]
food_groups.head()

In [ ]:
food_labels = pd.read_excel('../Input Data/ABBREV.xlsx', sheet_name='ABBREV')
food_labels.NDB_No = 'http://idea.rpi.edu/heals/kb/usda#' + food_labels.NDB_No.astype(str).str.rjust(5,'0')
food_labels = food_labels[['NDB_No', 'Shrt_Desc']]
food_labels.head()

In [ ]:
df1 = pd.merge(food_groups, food_labels, how='left', on='NDB_No')
df1 = df1.rename(columns = {'Shrt_Desc':'Food label'})
df1.head()

In [ ]:
df2 = pd.merge(nc, df1, how='left', on='Food label')
df2 = df2[['Food label', 'FdGrp_Desc']]
df2 = df2.drop_duplicates()
df2.head()

In [ ]:
df_ = pd.DataFrame()
df_['food_groups'] = list(Counter(df2['FdGrp_Desc']).keys())
df_['count'] = Counter(df2['FdGrp_Desc']).values()
df_ = df_.sort_values('food_groups', ascending=False)

In [ ]:
fig, ax = plt.subplots()
bars = ax.barh(df_['food_groups'], df_['count'], color='purple', alpha=0.5)

fig.set_size_inches(12, 8)

for i, v in enumerate(df_['count']):
    ax.text(v+1, i, str(v), fontweight='bold')
    
plt.ylabel('Food Categorization')
plt.xlabel('Count')
plt.show()